### Importations

In [1]:
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import json
import os
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import numpy as np

In [3]:
def recupTexteEntreBalise(texte, separateur):
    
    texte2 = []
    lisI = []
    lisS = []
    
    for i in range(0,len(texte)):
        if str(texte[i]) == "<":
            lisI.append(i)
        if texte[i] == '>':
            lisS.append(i)   

    taille = len(lisI)
    for h in range(0,taille):
        if h < (taille-1):
            texte2.append(texte[lisS[h]:lisI[h+1]])
    
    if separateur != 'non':
        description = str(texte2).replace('>','').replace(',','').replace('\'','').replace('，','')
        description = description.split(separateur)
    else:
        description = texte2
    
    return description

In [4]:
def supprimeBlanc(Author):
    Author2 = []
    for elem in Author:
        if len(elem) != 0:
            Author2.append(elem)
    return Author2

In [5]:
def recupWeibo(listURL):
    
    nom_col = ['Data_Source','Airline_Name','Airline_Type','Region_Operation','Aircraft_Type','Cabin_Class','Type_Of_Lounge',
               'Type_Of_Traveller','Date_Visit','Date_Flown','Airport','Route','Category','Category_Detail',
               'Cabin_Staff_Service','Lounge_Staff_Service','Bar_And_Beverages','Food_And_Beverages','Ground_Service',
               'Catering','Cleanliness','Lounge_Comfort','Aisle_Space','Wifi_And_Connectivity','Inflight_Entertainment',
               'Viewing_Tv_Screen','Power_Supply',
              'Seat','Seat_type','Seat_Comfort','Seat_Legroom','Seat_Storage','Seat_Width','Seat_Recline','Washrooms',
               'Value_For_Money','Overall_Customer_Rating','Overall_Service_Rating','Overall_Airline_Rating',
              'Recommended','Departure_city','Arrival_city','Nb_bus_taken','Nb_train_taken',
               'Nb_car_taken','Nb_plane_taken','Duration','Price_min','Price_max','Nb_sharing','Awards',
               'Registration','Language']
    df = pd.DataFrame(columns = nom_col)
 
    for i in listURL:
        
        r = requests.get(i)
        page = r.text
        soup = bs(page,'html.parser')
        
        # Description Post
        texte = soup.findAll('p',attrs={'node-type':'feed_list_content'})
        texte = str(texte)
        description = recupTexteEntreBalise(texte, '\\n')
        del description[0]
        
        description = np.array(description).T
 
        # Date Post
        date = soup.findAll('p',attrs={'class':'from'})
        date = str(date)
        date2 = recupTexteEntreBalise(texte, 'non')
        
        date2 = np.array(date2).T
        date2 = re.findall(r'[0-9]+年[0-9]+月[0-9]+日',str(date2))

        # Auteur Post
        auteur = soup.findAll('a',attrs={'class':'name'})
        auteur = str(auteur)
        Author = recupTexteEntreBalise(auteur, ' ')
        
        Author2 = supprimeBlanc(Author)
        
        
        # Partage / Commentaire, Avis / Like
        aime = soup.findAll('div',attrs={'class':"card-act"})
        aime = str(aime)
        aime2 = recupTexteEntreBalise(aime, 'non')
        
        Partage = []
        Comm = []
        Like = []

        for i in range(0,len(aime2)):
            if aime2[i].startswith('> 转发'):
                Partage.append(str(re.findall(r'[0-9]+',str(aime2[i]))).replace('[','').replace(']','').replace('\'',''))
            if aime2[i].startswith('>评论'):
                Comm.append(str(re.findall(r'[0-9]+',str(aime2[i]))).replace('[','').replace(']','').replace('\'',''))
            test = re.findall(r'>[0-9]+',aime2[i])
            if (str(test))!='[]':
                test = re.findall(r'[0-9]+',str(test))
                Like.append(str(test).replace('[','').replace(']','').replace('\'','').replace('>',''))
            else:
                if aime2[i-7].startswith('>评论'):
                    Like.append(str(aime2[i]).replace('[','').replace(']','').replace('\'','').replace('>',''))

        Comm = np.array(Comm).T
        Partage = np.array(Partage).T
        Like = np.array(Like).T
        
        s = pd.DataFrame(data=(description,Author2,Partage,Comm,Like))
        detailVideos = s.transpose()
        detailVideos.columns = ['Description','Author','Nb_sharing','Nb_comments','Likes']

        df = pd.concat([df, detailVideos])
    return df

In [24]:
compagniesAeriennes=["American Airlines","Air Canada","Air France","Air Algerie","Air India","Aerolineas Argentinas","Royal Air Maroc","Finnair" ,"Alitalia "," Nouvelair","Air China","Cathay Pacific","Delta Airlines","Aer Lingus","Emirates","Ethiopian Airlines","Icelandair","Hawaiian Airlines","Iberia","Meridiana","Japan Airlines","KLM","Air Malta","Lan Airlines","Luxair","LIAT","LOT Polish Airlines","Air Madagascar","Air Mauritius","Austrian Airlines","Qatar Airways","South African Airways","SAS Scandinavian Airlines","Brussels Airlines","Singapore Airlines","Corsair","Aeroflot","Thai Airways","Turkish Airlines","TAP Portugal","Air Transat","Tunisair","Air Caraibes","United Airlines","Air Austral","Air Europa","Easyjet","Vietnam Airlines","Virgin Atlantic","Air Corsica","Condor","Flybe","Aegean Airlines","Air Tahiti Nui","Aigle Azur","HOP!","Jet Airways","Etihad Airways","Etihad Airways","Oman Air","XL Airways","Ryanair LTD","Vueling ","Norwegian","Transavia France","Germanwings","TUI Fly Belgium","Air Arabia","WOW air","Wizz Air","Air Asia","Volotea","southwest airlines"]
modelesBoeing=["Boeing 717","Boeing 727","Boeing 737-200","Boeing 737-300","Boeing 737-400","Boeing 737-500","Boeing 737-600","Boeing 737-700","Boeing 737-700ER","Boeing 737-800","Boeing 737-900","Boeing 737-900ER","Boeing 737 MAX 7","Boeing 737 MAX 8","Boeing 737 MAX 9","Boeing 737 MAX 10","Boeing 747-200","Boeing 747-400","Boeing 757-200","Boeing 757-300","Boeing 767-200","Boeing 767-300","Boeing 767-300ER","Boeing 767-400ER","Boeing 777 Triple Seven","Boeing 787 DreamLiner"]
modelesAirbus=["A300","A300-600ST","A318","A319","A320-100","A320-200","A320neo","A321-100","A321-200","A330-200","A330-300","A330-200F","A330-500","A340-200","A340-300","A340-500","A340-600","A350-900","A350-1000","A380-800","A220-300"]
motsCles=["trip","fly","plane","airplane","flight"]

equations=[]
for comp in compagniesAeriennes:
    for mod in modelesAirbus:
        equations.append('https://s.weibo.com/weibo/'+comp+'%2520'+mod)
    for mod in modelesBoeing:
        equations.append('https://s.weibo.com/weibo/'+comp+'%2520'+mod)

In [25]:
len(equations)

1533

In [26]:
# Equation de recherche
liste1 = ['airbus', 'boeing']
liste2 = ['A320', 'A360']
a = []
for i in liste1:
    for j in liste2:
        a.append('https://s.weibo.com/weibo/'+i+'%2520'+j)

In [27]:
b = recupWeibo(a)
b

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Aircraft_Type,Airline_Name,Airline_Type,Airport,Aisle_Space,Arrival_city,Author,Awards,Bar_And_Beverages,Cabin_Class,...,Seat_Recline,Seat_Storage,Seat_Width,Seat_type,Type_Of_Lounge,Type_Of_Traveller,Value_For_Money,Viewing_Tv_Screen,Washrooms,Wifi_And_Connectivity
0,NaN,NaN,NaN,NaN,NaN,NaN,[三七V博,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,FATIII,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,中性灰,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Wingsworld_飞机模型,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,开飞机的LuSam-WUH,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Haiwe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,巧笑嫣然lan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,ICTJoseph,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,wings翱翔,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,kisar_m6518757848,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
c = b.to_json(orient='records')
with open('dataWeibo.json', 'a', encoding='utf8') as outfile:
        json.dump(c, outfile, ensure_ascii=False,indent=4)

In [26]:
r = requests.get('https://s.weibo.com/weibo/Airbus%2520A380')
page = r.text
soup = bs(page,'html.parser')

date = soup.findAll('p',attrs={'class':'from'})
date = str(date)
description = recupTexteEntreBalise(date, 'non')
description = np.array(description).T

In [29]:
re.findall(r'[0-9]+年[0-9]+月[0-9]+日',str(description))

['2019年11月23日',
 '2019年05月29日',
 '2019年04月09日',
 '2019年12月31日',
 '2019年12月28日',
 '2019年12月25日',
 '2019年12月21日',
 '2019年12月21日',
 '2019年12月19日',
 '2019年12月18日',
 '2019年12月16日',
 '2019年12月14日',
 '2019年12月13日',
 '2019年11月21日',
 '2019年11月04日',
 '2019年10月13日',
 '2019年09月28日',
 '2019年09月24日',
 '2019年09月18日',
 '2019年09月14日',
 '2019年09月06日',
 '2019年09月01日']